### Here I will try running RRCF parallely and create the final dataset that the agent would see

In [2]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from joblib import Parallel, delayed

In [3]:
current_dir = Path().cwd()
project_root = current_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print('Done')

Done


In [4]:
from src.sync.RRCF_TS_anomaly_detection import *
from src.utils.save_data import *
from src.sync.Feature_Transformation import feature_engineering, compute_speaker_median_pitch, get_speaker_timings

In [5]:
path = project_root / "data" / "processed" / "merge" / "final_smooth_rz_data.parquet"
df=load_df_parquet_safe(path=str(path))
path = project_root / "data" / "processed" / "merge" / "avw_merged_2.parquet"
non_normfe_df = load_df_parquet_safe(path=str(path))

In [6]:
df.head()

,Time,blink_intensity_smooth_rz,gaze_magnitude_smooth_rz,jaw_magnitude_smooth_rz,smile_intensity_smooth_rz,loudness_db_smooth_rz,pitch_relative_st_smooth_rz,pitch_expressiveness_st_smooth_rz,wps_smooth_rz,words,text_concat,speaker,filler_percentage,pause_percent_pr
0,0.0,3.571464,1.555960,-0.351862,1.418536,NaN,NaN,NaN,NaN,"[We're, starting]",We're starting,None,0.064433,0.079177
1,0.5,1.740830,1.323856,-0.425948,1.438164,NaN,NaN,NaN,NaN,"[now., [*]]",now. [*],A,0.064433,0.079177
2,1.0,1.297544,1.195807,-0.627946,1.056126,NaN,NaN,NaN,NaN,None,,A,0.064433,0.079177
3,1.5,1.004625,0.428483,-0.742188,0.602340,NaN,NaN,NaN,NaN,"[So, welcome]",So welcome,A,0.064433,0.079177
4,2.0,1.322258,1.213327,-0.828038,0.355089,NaN,NaN,NaN,NaN,"[to, the]",to the,A,0.064433,0.079177


In [7]:
feature_cols = [
    ('blink_intensity_smooth_rz', 'ui'),
    ('gaze_magnitude_smooth_rz', 'ui'),
    ('jaw_magnitude_smooth_rz', 'ui'),
    ('smile_intensity_smooth_rz', 'ui'),
    ('loudness_db_smooth_rz', 'ud'),
    ('pitch_relative_st_smooth_rz', 'ud'),
    ('pitch_expressiveness_st_smooth_rz', 'ud'),
    ('wps_smooth_rz', 'ud'),
    ('filler_percentage', 'ud'),
    ('pause_percent_pr', 'ud'),
]

features = {}
feat_id = {}
for name, type in feature_cols:
    f_id, vector = get_data_ready(data=df, features=[name], type=type)
    features[name] = vector
    feat_id[name] = f_id

In [8]:
for key, value in features.items():
    print(value)

[[3.57146417]
 [1.74083022]
 [1.29754446]
 ...
 [1.6549074 ]
 [1.252199  ]
 [1.04397284]]
[[1.55595991]
 [1.3238559 ]
 [1.19580709]
 ...
 [1.07019098]
 [0.47603773]
 [0.01574653]]
[[-0.35186165]
 [-0.42594774]
 [-0.62794575]
 ...
 [-0.90239459]
 [-0.90098368]
 [-0.89959585]]
[[ 1.41853557e+00]
 [ 1.43816411e+00]
 [ 1.05612625e+00]
 ...
 [ 2.40739112e-01]
 [ 8.45211635e-04]
 [-1.89532529e-01]]
[[-0.39149598]
 [-1.81300585]
 [-1.78566912]
 ...
 [ 0.50776887]
 [ 0.0872767 ]
 [-1.10199763]]
[[-0.67413378]
 [-0.69538963]
 [-0.62424157]
 ...
 [-0.66407274]
 [-1.09142559]
 [-0.93226954]]
[[ 0.3390234 ]
 [-0.34631684]
 [-1.44226786]
 ...
 [-0.53677324]
 [-0.70190335]
 [-1.69625822]]
[[-3.21992348]
 [-3.21992348]
 [-2.49393394]
 ...
 [ 1.21875057]
 [ 2.12852661]
 [ 0.60039801]]
[[ 0.06443299]
 [ 0.06443299]
 [ 0.06443299]
 ...
 [13.46649485]
 [13.46649485]
 [13.46649485]]
[[0.07917656]
 [0.07917656]
 [0.07917656]
 ...
 [9.58036421]
 [9.58036421]
 [9.65954078]]


In [9]:
results = Parallel(n_jobs=-1)(
    delayed(run_rrcf)(
        features=value,
        shingle=10
    )
    for _, value in features.items()
)

In [10]:
results

[[0.0,
  1.0000000000000004,
  1.2249999999999999,
  1.5499999999999992,
  1.0500000000000005,
  1.3499999999999999,
  2.3333333333333326,
  3.549999999999999,
  4.266666666666666,
  1.9924999999999995,
  1.6399999999999995,
  1.3279166666666666,
  1.6704166666666658,
  2.0470833333333336,
  6.966666666666669,
  7.143749999999998,
  8.759166666666665,
  5.754166666666666,
  4.320535714285715,
  3.084375,
  2.8965674603174616,
  2.1973809523809527,
  2.4247023809523807,
  2.446388888888889,
  3.145828823953823,
  1.6881394300144303,
  2.454839743589743,
  3.654166666666666,
  2.1291666666666664,
  3.527559523809523,
  2.4826515151515145,
  2.2958665639915647,
  2.3267640692640703,
  1.9587500000000002,
  2.756547619047619,
  4.133449883449884,
  2.5246098346098353,
  2.4555462592962596,
  2.704805194805195,
  4.28389423076923,
  1.2702083333333332,
  3.7690277777777776,
  2.4808196419593482,
  2.7314228279030903,
  2.1697650422118304,
  2.1765506715506713,
  1.8634995213120213,
  2.9493

In [11]:
anomaly_scores = {key:pd.DataFrame(value, index=feat_id[key], columns=['anomaly_scores']) for key, value in zip([feature[0] for feature in feature_cols], results)}

In [12]:
indexes = df.index

In [13]:
df[df['Time'] == 2.5].index.item()

5

In [14]:
anomaly_scores['blink_intensity_smooth_rz']['anomaly_scores']

0       0.000000
1       1.000000
2       1.225000
3       1.550000
4       1.050000
          ...   
1259    4.435168
1260    4.229185
1261    2.829870
1262    2.909416
1263    3.415251
Name: anomaly_scores, Length: 1264, dtype: float64

In [15]:
for key, value in anomaly_scores.items():
    n_sigma = adaptive_n_sigma(anomaly_scores=value['anomaly_scores'])
    # print(n_sigma)
    threshold = get_threshold_mad(scores=value['anomaly_scores'], n_sigma=n_sigma)
    # print(threshold)
    value['is_anomaly'] = value['anomaly_scores'] > threshold
    
    value.loc[value[value['is_anomaly']].index, ['anomaly_index']] = value[value['is_anomaly']].index.map(lambda x: int(indexes.get_loc(x)))
    # value['anomaly_index'] = value['anomaly_index'].astype("Int64")

In [16]:
anomaly_scores

{'blink_intensity_smooth_rz':       anomaly_scores  is_anomaly  anomaly_index
 0           0.000000       False            NaN
 1           1.000000       False            NaN
 2           1.225000       False            NaN
 3           1.550000       False            NaN
 4           1.050000       False            NaN
 ...              ...         ...            ...
 1259        4.435168       False            NaN
 1260        4.229185       False            NaN
 1261        2.829870       False            NaN
 1262        2.909416       False            NaN
 1263        3.415251       False            NaN
 
 [1264 rows x 3 columns],
 'gaze_magnitude_smooth_rz':       anomaly_scores  is_anomaly  anomaly_index
 0           0.000000       False            NaN
 1           1.000000       False            NaN
 2           1.325000       False            NaN
 3           2.600000       False            NaN
 4           1.025000       False            NaN
 ...              ...         ...

In [17]:
anomaly_scores['blink_intensity_smooth_rz']['is_anomaly'] == True

0       False
1       False
2       False
3       False
4       False
        ...  
1259    False
1260    False
1261    False
1262    False
1263    False
Name: is_anomaly, Length: 1264, dtype: bool

In [18]:
c_anomalous = {}

for key, value in anomaly_scores.items():

    anomalous_indices = value[value['is_anomaly'] == True].index
    
    # 2. Create the dataframe with ONLY the Time column
    pass_df = df.loc[anomalous_indices, ['Time']].copy()
    
    # 3. Get the ranges (The function now reads 'Time' directly)
    c_anomalous[key] = get_anomalous_time_ranges(anomalies_time=pass_df)

In [19]:
c_anomalous

{'blink_intensity_smooth_rz': [[39.0, 39.5, 40.0],
  [147.5, 148.0, 149.0],
  [166.0, 166.5],
  [293.0, 295.0],
  [301.0, 302.0],
  [406.5, 407.5, 408.0, 408.5, 409.0, 409.5, 410.0, 410.5, 411.0],
  [426.0, 426.5],
  [437.5, 438.0, 439.0, 439.5, 440.0, 442.0],
  [572.5, 573.0],
  [576.0, 576.5, 577.0, 577.5, 579.0, 580.0],
  [598.0, 598.5, 599.0, 599.5]],
 'gaze_magnitude_smooth_rz': [[74.0, 74.5, 75.0, 75.5, 76.0],
  [129.0, 129.5, 130.0, 131.5],
  [165.0, 165.5, 166.0, 166.5],
  [193.0, 193.5, 194.0],
  [212.0, 212.5, 213.0, 213.5, 214.0, 214.5],
  [235.5, 236.0, 236.5, 237.0, 237.5, 238.0],
  [255.5, 256.5, 257.0, 257.5, 258.0, 260.0, 260.5],
  [284.5, 285.0, 285.5, 286.0, 286.5, 287.0],
  [396.5, 397.5, 398.0, 399.5, 400.0, 400.5, 401.0],
  [478.5, 479.0],
  [491.5, 493.5],
  [530.5, 531.0, 531.5],
  [536.5, 537.0, 537.5, 538.0],
  [582.0, 582.5],
  [601.0,
   603.0,
   604.0,
   604.5,
   605.0,
   605.5,
   606.0,
   606.5,
   607.0,
   607.5,
   608.0,
   610.0],
  [619.5, 620.0

In [20]:
anomaly_scores['blink_intensity_smooth_rz']['anomaly_index']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1259   NaN
1260   NaN
1261   NaN
1262   NaN
1263   NaN
Name: anomaly_index, Length: 1264, dtype: float64

In [21]:
# anomalies = {key: list(value['anomaly_index'][value['anomaly_index'].notna()]) for key, value in anomaly_scores.items()}

# Create 'anomalies' using TIME, not Index
anomalies = {}

for key, value in anomaly_scores.items():
    # Filter for rows where is_anomaly is True
    anomalous_rows = value[value['is_anomaly'] == True]
    
    # Extract the corresponding Time from the main df
    # We use the index of the anomalous rows to look up the Time in 'df'
    anomalous_times = df.loc[anomalous_rows.index, 'Time'].tolist()
    
    anomalies[key] = anomalous_times

# Now 'anomalies' contains [23.0, 23.5, ...] instead of [46, 47, ...]

In [22]:
# anomalies = {key: list(map(int, value)) for key, value in anomalies}
anomalies

{'blink_intensity_smooth_rz': [24.5,
  34.5,
  39.0,
  39.5,
  40.0,
  67.5,
  70.5,
  106.0,
  122.0,
  138.5,
  147.5,
  148.0,
  149.0,
  166.0,
  166.5,
  186.5,
  220.0,
  230.5,
  240.5,
  287.5,
  293.0,
  295.0,
  301.0,
  302.0,
  312.0,
  388.0,
  406.5,
  407.5,
  408.0,
  408.5,
  409.0,
  409.5,
  410.0,
  410.5,
  411.0,
  426.0,
  426.5,
  437.5,
  438.0,
  439.0,
  439.5,
  440.0,
  442.0,
  478.0,
  483.5,
  518.5,
  531.0,
  544.5,
  557.5,
  562.0,
  565.5,
  572.5,
  573.0,
  576.0,
  576.5,
  577.0,
  577.5,
  579.0,
  580.0,
  585.5,
  598.0,
  598.5,
  599.0,
  599.5,
  606.0,
  618.5],
 'gaze_magnitude_smooth_rz': [74.0,
  74.5,
  75.0,
  75.5,
  76.0,
  129.0,
  129.5,
  130.0,
  131.5,
  143.0,
  165.0,
  165.5,
  166.0,
  166.5,
  193.0,
  193.5,
  194.0,
  202.0,
  212.0,
  212.5,
  213.0,
  213.5,
  214.0,
  214.5,
  227.0,
  235.5,
  236.0,
  236.5,
  237.0,
  237.5,
  238.0,
  255.5,
  256.5,
  257.0,
  257.5,
  258.0,
  260.0,
  260.5,
  280.0,
  284.5,


In [23]:
non_normfe_df.columns

Index(['Time', 'words', 'text_concat', 'h_ratio', 'v_ratio', '_neutral',
       'browDownLeft', 'browDownRight', 'browInnerUp', 'browOuterUpLeft',
       'browOuterUpRight', 'cheekPuff', 'cheekSquintLeft', 'cheekSquintRight',
       'eyeBlinkLeft', 'eyeBlinkRight', 'eyeLookDownLeft', 'eyeLookDownRight',
       'eyeLookInLeft', 'eyeLookInRight', 'eyeLookOutLeft', 'eyeLookOutRight',
       'eyeLookUpLeft', 'eyeLookUpRight', 'eyeSquintLeft', 'eyeSquintRight',
       'eyeWideLeft', 'eyeWideRight', 'jawForward', 'jawLeft', 'jawOpen',
       'jawRight', 'mouthClose', 'mouthDimpleLeft', 'mouthDimpleRight',
       'mouthFrownLeft', 'mouthFrownRight', 'mouthFunnel', 'mouthLeft',
       'mouthLowerDownLeft', 'mouthLowerDownRight', 'mouthPressLeft',
       'mouthPressRight', 'mouthPucker', 'mouthRight', 'mouthRollLower',
       'mouthRollUpper', 'mouthShrugLower', 'mouthShrugUpper',
       'mouthSmileLeft', 'mouthSmileRight', 'mouthStretchLeft',
       'mouthStretchRight', 'mouthUpperUpLeft', 'mo

In [24]:
cols = ['h_ratio', 'v_ratio', '_neutral',
       'browDownLeft', 'browDownRight', 'browInnerUp', 'browOuterUpLeft',
       'browOuterUpRight', 'cheekPuff', 'cheekSquintLeft', 'cheekSquintRight',
       'eyeBlinkLeft', 'eyeBlinkRight', 'eyeLookDownLeft', 'eyeLookDownRight',
       'eyeLookInLeft', 'eyeLookInRight', 'eyeLookOutLeft', 'eyeLookOutRight',
       'eyeLookUpLeft', 'eyeLookUpRight', 'eyeSquintLeft', 'eyeSquintRight',
       'eyeWideLeft', 'eyeWideRight', 'jawForward', 'jawLeft', 'jawOpen',
       'jawRight', 'mouthClose', 'mouthDimpleLeft', 'mouthDimpleRight',
       'mouthFrownLeft', 'mouthFrownRight', 'mouthFunnel', 'mouthLeft',
       'mouthLowerDownLeft', 'mouthLowerDownRight', 'mouthPressLeft',
       'mouthPressRight', 'mouthPucker', 'mouthRight', 'mouthRollLower',
       'mouthRollUpper', 'mouthShrugLower', 'mouthShrugUpper',
       'mouthSmileLeft', 'mouthSmileRight', 'mouthStretchLeft',
       'mouthStretchRight', 'mouthUpperUpLeft', 'mouthUpperUpRight',
       'noseSneerLeft', 'noseSneerRight']

In [25]:
non_normfe_df[cols] = non_normfe_df[cols].ffill()

In [26]:
non_normfe_df[non_normfe_df['jawOpen'].isna()]

,Time,words,text_concat,h_ratio,v_ratio,_neutral,browDownLeft,browDownRight,browInnerUp,browOuterUpLeft,...,mouthUpperUpRight,noseSneerLeft,noseSneerRight,audio_rms(volumn),audio_pitch_avg,audio_pitch_var(expressiveness),is_silent,speaker,filler_percentage,pause_percent_pr


In [27]:
speaker_timmings = get_speaker_timings(speaker_times=df[['Time', 'speaker']], speaker='B')

In [28]:
audio_path = project_root / "data" / "raw" / "Interview_2.wav"
median = compute_speaker_median_pitch(audio_path=str(audio_path), speaker_segments=speaker_timmings)

In [29]:
median = 158.74

In [30]:
final_dataframe = feature_engineering(mode="evaluation", c_anomalies=c_anomalous, anomalies=anomalies, df=non_normfe_df, norm_rz_df=df, speaker_median_pitch=median, speaker='B')

In [31]:
final_dataframe.iloc[20:40]

,blinking_data,gaze_data,jaw_movement_data,smile_data,loudness_data,average_pitch_data,pitch_standard_deviation,words_per_sec,filler_words_usage,pauses_taken
20,"{'intensity': 0.056107991188764576, 'asymmetry...","{'horizontal_deviation': 0.08830405917372364, ...","{'open': 0.0003300155804026872, 'lateral': -0....","{'intensity': 1.970937121775762e-05, 'asymmetr...",NaN,NaN,NaN,NaN,NaN,NaN
21,"{'intensity': 0.06253535598516465, 'asymmetry'...","{'horizontal_deviation': 0.07903441936489153, ...","{'open': 9.470622899243608e-05, 'lateral': -0....","{'intensity': 0.0026545619226879145, 'asymmetr...",NaN,NaN,NaN,NaN,NaN,NaN
22,"{'intensity': 0.40232221484184266, 'asymmetry'...","{'horizontal_deviation': 0.13410172586128022, ...","{'open': 0.06632930040359497, 'lateral': -0.00...","{'intensity': 5.0152379290580026e-05, 'asymmet...",NaN,NaN,NaN,NaN,NaN,NaN
23,"{'intensity': 0.1325589820742607, 'asymmetry':...","{'horizontal_deviation': 0.21401673924876086, ...","{'open': 0.0005459068925119936, 'lateral': 0.0...","{'intensity': 2.1747304715802328e-05, 'asymmet...","{'level': 'normal', 'rz_score': -0.39149597783...","{'relative_level': 'normal', 'rz_score': -0.67...","{'expressiveness': 'expressive', 'rz_score': 0...","{'speaking_rate': 'very_slow', 'rz_score': -3....","{'filler_percentage_level': 'normal', 'is_anom...","{'pause_percentage_level': 'normal', 'is_anoma..."
24,"{'intensity': 0.09494758993387223, 'asymmetry'...","{'horizontal_deviation': 0.22843528279342307, ...","{'open': 0.0005173762328922749, 'lateral': 0.0...","{'intensity': 1.3380005162844098e-05, 'asymmet...","{'level': 'normal', 'rz_score': -1.81300585287...","{'relative_level': 'normal', 'rz_score': -0.69...","{'expressiveness': 'expressive', 'rz_score': -...","{'speaking_rate': 'very_slow', 'rz_score': -3....","{'filler_percentage_level': 'normal', 'is_anom...","{'pause_percentage_level': 'normal', 'is_anoma..."
25,"{'intensity': 0.059617023915052414, 'asymmetry...","{'horizontal_deviation': 0.28380422869999267, ...","{'open': 0.018747227266430855, 'lateral': 0.00...","{'intensity': 1.3724788826152689e-05, 'asymmet...","{'level': 'normal', 'rz_score': -1.78566912450...","{'relative_level': 'normal', 'rz_score': -0.62...","{'expressiveness': 'slightly_expressive', 'rz_...","{'speaking_rate': 'slow', 'rz_score': -2.49393...","{'filler_percentage_level': 'normal', 'is_anom...","{'pause_percentage_level': 'normal', 'is_anoma..."
26,"{'intensity': 0.2767486721277237, 'asymmetry':...","{'horizontal_deviation': -0.05024080645921647,...","{'open': 0.041224658489227295, 'lateral': 0.00...","{'intensity': 1.0467591879148584e-05, 'asymmet...","{'level': 'normal', 'rz_score': -1.13186570441...","{'relative_level': 'normal', 'rz_score': -0.46...","{'expressiveness': 'slightly_expressive', 'rz_...","{'speaking_rate': 'normal', 'rz_score': -1.249...","{'filler_percentage_level': 'normal', 'is_anom...","{'pause_percentage_level': 'normal', 'is_anoma..."
27,"{'intensity': 0.24788355231285097, 'asymmetry'...","{'horizontal_deviation': 0.0168583169468286, '...","{'open': 0.09769998490810394, 'lateral': -0.00...","{'intensity': 6.6711254877560576e-06, 'asymmet...","{'level': 'normal', 'rz_score': -1.11288186527...","{'relative_level': 'normal', 'rz_score': -0.57...","{'expressiveness': 'slightly_expressive', 'rz_...","{'speaking_rate': 'normal', 'rz_score': -0.360...","{'filler_percentage_level': 'normal', 'is_anom...","{'pause_percentage_level': 'normal', 'is_anoma..."
28,"{'intensity': 0.16270604580640793, 'asymmetry'...","{'horizontal_deviation': 0.04744213935885999, ...","{'open': 0.14945782721042633, 'lateral': -0.06...","{'intensity': 6.139134125859868e-06, 'asymmetr...","{'level': 'normal', 'rz_score': -0.80066099083...","{'relative_level': 'normal', 'rz_score': -0.69...","{'expressiveness': 'slightly_expressive', 'rz_...","{'speaking_rate': 'normal', 'rz_score': 0.2745...","{'filler_percentage_level': 'normal', 'is_anom...","{'pause_percentage_level': 'normal', 'is_anoma..."

In [32]:
non_normfe_df['Time'].iloc[c_anomalous['gaze_magnitude_smooth_rz'][0]]

74    37.0
74    37.0
75    37.5
75    37.5
76    38.0
Name: Time, dtype: float64

In [33]:
non_normfe_df.columns

Index(['Time', 'words', 'text_concat', 'h_ratio', 'v_ratio', '_neutral',
       'browDownLeft', 'browDownRight', 'browInnerUp', 'browOuterUpLeft',
       'browOuterUpRight', 'cheekPuff', 'cheekSquintLeft', 'cheekSquintRight',
       'eyeBlinkLeft', 'eyeBlinkRight', 'eyeLookDownLeft', 'eyeLookDownRight',
       'eyeLookInLeft', 'eyeLookInRight', 'eyeLookOutLeft', 'eyeLookOutRight',
       'eyeLookUpLeft', 'eyeLookUpRight', 'eyeSquintLeft', 'eyeSquintRight',
       'eyeWideLeft', 'eyeWideRight', 'jawForward', 'jawLeft', 'jawOpen',
       'jawRight', 'mouthClose', 'mouthDimpleLeft', 'mouthDimpleRight',
       'mouthFrownLeft', 'mouthFrownRight', 'mouthFunnel', 'mouthLeft',
       'mouthLowerDownLeft', 'mouthLowerDownRight', 'mouthPressLeft',
       'mouthPressRight', 'mouthPucker', 'mouthRight', 'mouthRollLower',
       'mouthRollUpper', 'mouthShrugLower', 'mouthShrugUpper',
       'mouthSmileLeft', 'mouthSmileRight', 'mouthStretchLeft',
       'mouthStretchRight', 'mouthUpperUpLeft', 'mo

In [34]:
len(final_dataframe), len(non_normfe_df)

(1264, 1264)

In [35]:
final_dataframe[['Time', 'words', 'text_concat', 'speaker']] = non_normfe_df[['Time', 'words', 'text_concat', 'speaker']]

In [36]:
final_dataframe

,blinking_data,gaze_data,jaw_movement_data,smile_data,loudness_data,average_pitch_data,pitch_standard_deviation,words_per_sec,filler_words_usage,pauses_taken,Time,words,text_concat,speaker
0,"{'intensity': 0.5590226024389267, 'asymmetry':...","{'horizontal_deviation': -0.03615564603520849,...","{'open': 0.023826630786061287, 'lateral': -0.0...","{'intensity': 0.00020072988331669704, 'asymmet...",NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[We're, starting]",We're starting,None
1,"{'intensity': 0.20927320420742035, 'asymmetry'...","{'horizontal_deviation': -0.08192594102298828,...","{'open': 0.01953260414302349, 'lateral': -0.00...","{'intensity': 0.00020793307342685806, 'asymmet...",NaN,NaN,NaN,NaN,NaN,NaN,0.5,"[now., [*]]",now. [*],A
2,"{'intensity': 0.2994565159082413, 'asymmetry':...","{'horizontal_deviation': -0.03495631815543154,...","{'open': 0.0008870773017406464, 'lateral': -0....","{'intensity': 6.213216343535776e-05, 'asymmetr...",NaN,NaN,NaN,NaN,NaN,NaN,1.0,None,,A
3,"{'intensity': 0.28583882749080664, 'asymmetry'...","{'horizontal_deviation': 0.005119363319349368,...","{'open': 0.0015012087533250451, 'lateral': -0....","{'intensity': 4.647151249770331e-06, 'asymmetr...",NaN,NaN,NaN,NaN,NaN,NaN,1.5,"[So, welcome]",So welcome,A
4,"{'intensity': 0.37450549006462097, 'asymmetry'...","{'horizontal_deviation': 0.0903685688076663, '...","{'open': 0.00043307070154696703, 'lateral': -0...","{'intensity': 4.343396951611567e-05, 'asymmetr...",NaN,NaN,NaN,NaN,NaN,NaN,2.0,"[to, the]",to the,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,"{'intensity': 0.34180927872657774, 'asymmetry'...",{'horizontal_deviation': -0.031796652791948876...,"{'open': 0.0018719969084486365, 'lateral': -0....","{'intensity': 2.4302891581129413e-05, 'asymmet...",NaN,NaN,NaN,NaN,NaN,NaN,629.5,[So],So,A
1260,"{'intensity': 0.4298533320426941, 'asymmetry':...","{'horizontal_deviation': -0.03508123437997779,...","{'open': 0.0018371319165453315, 'lateral': -0....","{'intensity': 2.613913233808773e-05, 'asymmetr...",NaN,NaN,NaN,NaN,NaN,NaN,630.0,"[yeah,]","yeah,",A
1261,"{'intensity': 0.37084557414054875, 'asymmetry'...",{'horizontal_deviation': -0.024139688294741113...,"{'open': 0.001750032533891499, 'lateral': -0.0...","{'intensity': 2.566660534313314e-05, 'asymmetr...",NaN,NaN,NaN,NaN,NaN,NaN,630.5,"[that's, a]",that's a,A
1262,"{'intensity': 0.2990010350942612, 'asymmetry':...",{'horizontal_deviation': -0.018448774624961306...,"{'open': 0.0019191708415746689, 'lateral': -0....","{'intensity': 1.664540692782879e-05, 'asymmetr...",NaN,NaN,NaN,NaN,NaN,NaN,631.0,"[really, solid, approach.]",really solid approach.,A


In [37]:
path = project_root / "data" / "processed" / "merge" / "final_dataframe.parquet"
save_df_parquet_safe(df=final_dataframe, path=str(path))

Saved Parquet: c:\Users\BIT\Desktop\mmr\data\processed\merge\final_dataframe.parquet
Saved schema: c:\Users\BIT\Desktop\mmr\data\processed\merge\final_dataframe.parquet.schema.json
